# DNN with Hyper-Parameters Tuning

In [13]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # disable GPU
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout

#hyper-parameter tuning imports
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import time
LOG_DIR = f"{int(time.time())}" # folder names as a timestamp

SEED = 111 # constant seed for reproducibility
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

df = pd.read_csv("UFC_TRAIN.csv")

# tackling imbalance issue
theMin = df["Winner"].value_counts().min()
minority = df[df["Winner"]==1].iloc[0:theMin]
undersampleMaj = df[df["Winner"]==0].iloc[0:theMin]
df = pd.concat([minority, undersampleMaj], axis=0)
print(df["Winner"].value_counts())

# train/test split
X = df.drop(["Winner","B_fighter","R_fighter"], axis=1).values
y = df["Winner"].values
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=SEED)

# scaling
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print(f"X_train shape: {X_train.shape} | X_test shape: {X_test.shape} | y_train shape: {y_train.shape} | y_test shape: {y_test.shape}")

# Parameters
MIN = 30
MAX = 256
STEP = 16
MAX_TRIALS = 2
EXE_PER_TRIAL = 1
EPOCHS = 10
PATIENCE = 16

# function to build the model (argument: hyper-parameter)
def build_model(hp):
    # model
    model = Sequential()
    
    # first layer's no. of neurons = hp.Int range of values to test
    model.add(Dense(hp.Int("input_units", min_value=MIN, max_value=MAX, step=STEP), activation='relu'))
    
    # range of 1 to 10 layers to test
    for i in range(hp.Int("no. Of Hidden Layers", 1, 5)):
        # for each added layer, again test range of neurons and add dropout
        model.add(Dense(hp.Int(f"Hidden_layer_{i+1}_units", min_value=MIN, max_value=MAX, step=STEP), activation='relu'))
        model.add(Dropout(0.5))
    
    # output layer
    model.add(Dense(units=1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

tuner = RandomSearch(
        build_model, # name of the function that builds the model
        objective="val_accuracy", # the thing that we're interested to trace
        max_trials = MAX_TRIALS, # no. of combinations to try
        executions_per_trial = EXE_PER_TRIAL, # no. of times to train each combination (true avg)
        directory=LOG_DIR) # directory to save the outputs

# prevent divergence of loss & val_loss via early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=PATIENCE)

1    1336
0    1336
Name: Winner, dtype: int64
X_train shape: (2137, 42) | X_test shape: (535, 42) | y_train shape: (2137,) | y_test shape: (535,)


In [14]:
tuner.search(x=X_train,
             y=y_train,
             epochs = EPOCHS,
#             batch_size = 64,
             callbacks=[early_stop],
             validation_data=(X_test,y_test))

Train on 2137 samples, validate on 535 samples
Epoch 1/10
2137/2137 [==============================] - ETA: 2:52 - loss: 0.6922 - accuracy: 0.53 - ETA: 33s - loss: 0.6956 - accuracy: 0.5063 - ETA: 15s - loss: 0.6974 - accuracy: 0.506 - ETA: 11s - loss: 0.6990 - accuracy: 0.495 - ETA: 8s - loss: 0.6962 - accuracy: 0.500 - ETA: 6s - loss: 0.6956 - accuracy: 0.50 - ETA: 4s - loss: 0.6960 - accuracy: 0.49 - ETA: 3s - loss: 0.6965 - accuracy: 0.48 - ETA: 3s - loss: 0.6957 - accuracy: 0.49 - ETA: 2s - loss: 0.6959 - accuracy: 0.50 - ETA: 1s - loss: 0.6957 - accuracy: 0.50 - ETA: 1s - loss: 0.6958 - accuracy: 0.50 - ETA: 1s - loss: 0.6954 - accuracy: 0.50 - ETA: 0s - loss: 0.6953 - accuracy: 0.49 - ETA: 0s - loss: 0.6954 - accuracy: 0.50 - ETA: 0s - loss: 0.6952 - accuracy: 0.49 - 4s 2ms/sample - loss: 0.6953 - accuracy: 0.4988 - val_loss: 0.6930 - val_accuracy: 0.5084
Epoch 2/10
2137/2137 [==============================] - ETA: 1s - loss: 0.7122 - accuracy: 0.40 - ETA: 1s - loss: 0.7096 - ac

Train on 2137 samples, validate on 535 samples
Epoch 1/10
2137/2137 [==============================] - ETA: 3:48 - loss: 0.7423 - accuracy: 0.31 - ETA: 55s - loss: 0.7047 - accuracy: 0.4609 - ETA: 30s - loss: 0.7017 - accuracy: 0.473 - ETA: 20s - loss: 0.6969 - accuracy: 0.484 - ETA: 15s - loss: 0.6938 - accuracy: 0.500 - ETA: 13s - loss: 0.6927 - accuracy: 0.504 - ETA: 10s - loss: 0.6941 - accuracy: 0.503 - ETA: 9s - loss: 0.6953 - accuracy: 0.501 - ETA: 8s - loss: 0.6966 - accuracy: 0.49 - ETA: 7s - loss: 0.6981 - accuracy: 0.49 - ETA: 6s - loss: 0.6979 - accuracy: 0.49 - ETA: 5s - loss: 0.7001 - accuracy: 0.48 - ETA: 4s - loss: 0.7001 - accuracy: 0.49 - ETA: 4s - loss: 0.7023 - accuracy: 0.48 - ETA: 3s - loss: 0.7030 - accuracy: 0.48 - ETA: 3s - loss: 0.7029 - accuracy: 0.48 - ETA: 2s - loss: 0.7018 - accuracy: 0.48 - ETA: 2s - loss: 0.7018 - accuracy: 0.48 - ETA: 2s - loss: 0.7004 - accuracy: 0.48 - ETA: 1s - loss: 0.7000 - accuracy: 0.48 - ETA: 1s - loss: 0.6997 - accuracy: 0.48 -

INFO:tensorflow:Oracle triggered exit


In [15]:
print(tuner.results_summary()) # top 10 trials
print(tuner.get_best_hyperparameters()[0].values) #values of best hyper-parameters
#print(tuner.get_best_models()[0].summary())

None
{'input_units': 78, 'no. Of Hidden Layers': 8, 'Hidden_layer_1_units': 222, 'Hidden_layer_2_units': 30, 'Hidden_layer_3_units': 30, 'Hidden_layer_4_units': 30, 'Hidden_layer_5_units': 30, 'Hidden_layer_6_units': 30, 'Hidden_layer_7_units': 30, 'Hidden_layer_8_units': 30}
